In [12]:

import numpy as np
import numpy.random as ndm
import torch
from torch import nn
import pandas as pd
#import matplotlib.pyplot as plt
from data_generator import generate_case_3
from g_deep import g_D
from CCP_estimation import CCP_est
from iteration_dcp import Est_dcp
from iteration_dcp import Est_dcp1
from I_spline import I_S
from Least_FD import LFD
from g_dcp import g_DCP1
from Theta_estimate import Theta_est
from zeta_estimate import zeta_est

In [11]:
def set_seed(seed):
    np.random.seed(seed) 
    torch.manual_seed(seed) 

set_seed(114514)
tau = 10
p = 3 
n = 2000
corr = 0.5 
n_layer = 3
n_node = 50
n_epoch = 200 #
Set_lr = np.array([2.8e-4, 3.2e-4, 4.2e-4]) #learning rate
n_lr = 3.2e-4
Theta = [-1, 1]
zeta = 2

#dim_x = X_test.shape[0]
u_value = np.array(np.linspace(0, tau, 50), dtype="float32") 
Lambda_true = np.sqrt(u_value)/5 
m = 10 
nodevec = np.array(np.linspace(0, tau, m+2), dtype="float32")

m0 = 4 
nodevec0 = np.array(np.linspace(0, 2, m0+2), dtype="float32")

node_D = np.array([35, 30, 30])   #deep
lr_D = np.array([4e-4, 4e-4, 4e-4])

c0 = np.array(0.1*np.ones(m+p), dtype="float32") 
Theta0 = np.array([0,0], dtype='float32')
       

In [3]:
train_data = generate_case_3(n, corr, Theta, zeta)
Z_train = train_data['Z']
X_train = train_data['X']
U_train = train_data['U']
De_train = train_data['De']
g_train = train_data['g_X']
g_train_true = train_data['g_X']
Z_2_train = train_data['Z_2']
Lambda_U = I_S(m,c0,U_train,nodevec)

In [13]:
test_data = generate_case_3(200, corr, Theta, zeta)
X_test = test_data['X']
g_true = test_data['g_X']
Z_2_test = test_data['Z_2']
#print(X_test.shape)

In [5]:

#Theta0 = np.array([-0.9,0.9], dtype='float32')
n_lr = 1.5e-4
n_epoch = 200 #
#Z_train = train_data['Z']
#U_train = train_data['U']
#De_train = train_data['De']


#C_index = 0
#zeta0 = np.mean(Z_2_train)
#Z_train = train_data['Z']
#Z_2_train = train_data['Z_2']
#ZC = np.vstack((Z_train,Z_train*(Z_2_train>zeta)))
#locx_tra1 = np.where(Z_2_train>zeta)  #Z_2>zeta in train data
#locx_tra0 = np.where(Z_2_train<=zeta)
#locx_tes1 = np.where(Z_2_test>zeta)  #Z_2>zeta in test data
#locx_tes0 = np.where(Z_2_test<=zeta)
#ZC0 = ZC.T[locx_tra0]
#ZC1 = ZC.T[locx_tra1]
#print(locx_tra1)
#print(ZC.shape)
#print(ZC1.shape)
#ZC1 = torch.Tensor(ZC1)
#print(ZC1.T@Theta0)

In [6]:
#one single experiment 
zeta0 = np.mean(Z_2_train)
result = Est_dcp(train_data,X_test,Z_2_test,Theta,Theta0,zeta0,n_layer,n_node,n_lr,n_epoch,nodevec,m,c0)
Theta_res = result['Theta']
zeta_res = result['zeta']
C_res = result['C_index']
g_test_res = result['g_test']
error = (g_test_res-g_true)@(g_test_res.T-g_true.T)/200
print(Theta_res)
print(zeta_res)
print(C_res)
print(error)

[-0.78144489  0.63477454]
[1.8]
1
5.7310308079310825


In [9]:
zeta0 = np.mean(Z_2_train)
result = Est_dcp1(train_data,X_test,Z_2_test,Theta,Theta0,zeta0,n_layer,n_node,n_lr,n_epoch,nodevec,m,c0)
Theta_res = result['Theta']
zeta_res = result['zeta']
C_res = result['C_index']
g_test_res = result['g_test']
error = (g_test_res-g_true)@(g_test_res.T-g_true.T)/200
print(Theta_res)
print(zeta_res)
print(C_res)
print(error)

[-0.7908739   0.78279771]
[2.]
0
22.72442138671875


In [14]:
# i.i.d. experiments
G_test_deep = []
ThetaM = []
zetaM = []
for b in range(50):
    print('n=', n, 'b=', b)
    set_seed(12 + b)
    c0 = np.array(0.1*np.ones(m+p), dtype="float32")
    Theta0 = np.array([-0.9,0.9], dtype='float32')
    train_data = generate_case_3(n, corr, Theta, zeta)
    Z_train = train_data['Z']
    U_train = train_data['U']
    De_train = train_data['De']
    g_train = train_data['g_X']
    Z_2_train = train_data['Z_2']
    zeta0 = np.mean(Z_2_train)
    result = Est_dcp(train_data,X_test,Z_2_test,Theta,Theta0,zeta0,n_layer,n_node,n_lr,n_epoch,nodevec,m,c0)
    Theta_res = result['Theta'] # vector to add row by row
    ThetaM.append(Theta_res)
    zeta_res = result['zeta']
    zetaM.append(zeta_res)
    #C_res = result['C_index']
    g_test_res = result['g_test']
    G_test_deep.append(result['g_test']) # vector to add row by row
Error_dcp = np.mean(np.array(G_test_deep), axis=0) - g_true
Theta_dcp = np.mean(np.array(ThetaM),axis=0)
zeta_dcp = np.mean(np.array(zeta_res))
print(Error_dcp)
print(Theta_dcp)
print(zeta_dcp)

n= 2000 b= 0
n= 2000 b= 1
n= 2000 b= 2
n= 2000 b= 3
n= 2000 b= 4
n= 2000 b= 5
n= 2000 b= 6
n= 2000 b= 7
n= 2000 b= 8
n= 2000 b= 9
n= 2000 b= 10
n= 2000 b= 11
n= 2000 b= 12
n= 2000 b= 13
n= 2000 b= 14
n= 2000 b= 15
n= 2000 b= 16
n= 2000 b= 17
n= 2000 b= 18
n= 2000 b= 19
n= 2000 b= 20
n= 2000 b= 21
n= 2000 b= 22
n= 2000 b= 23
n= 2000 b= 24
n= 2000 b= 25
n= 2000 b= 26
n= 2000 b= 27
n= 2000 b= 28
n= 2000 b= 29
n= 2000 b= 30
n= 2000 b= 31
n= 2000 b= 32
n= 2000 b= 33
n= 2000 b= 34
n= 2000 b= 35
n= 2000 b= 36
n= 2000 b= 37
n= 2000 b= 38
n= 2000 b= 39
n= 2000 b= 40
n= 2000 b= 41
n= 2000 b= 42
n= 2000 b= 43
n= 2000 b= 44
n= 2000 b= 45
n= 2000 b= 46
n= 2000 b= 47
n= 2000 b= 48
n= 2000 b= 49
[  1.49247169   2.5394449   -1.29824762   1.84308771   1.48930824
   0.17767249   0.80951703   2.68154692   1.53811856   1.94238612
  -0.5009684    1.05519675   1.47052424   2.28878242   2.68011795
   0.97208465  -0.21940189   0.37936425   1.4906825    1.36540966
  -4.42117788  -3.38643939  -1.96492595   1.49

In [ ]:
G_test_deep = []
ThetaM = []
zetaM = []
for b in range(50):
    print('n=', n, 'b=', b)
    set_seed(12 + b)
    c0 = np.array(0.1*np.ones(m+p), dtype="float32")
    Theta0 = np.array([-0.9,0.9], dtype='float32')
    train_data = generate_case_3(n, corr, Theta, zeta)
    Z_train = train_data['Z']
    U_train = train_data['U']
    De_train = train_data['De']
    g_train = train_data['g_X']
    Z_2_train = train_data['Z_2']
    zeta0 = np.mean(Z_2_train)
    result = Est_dcp1(train_data,X_test,Z_2_test,Theta,Theta0,zeta0,n_layer,n_node,n_lr,n_epoch,nodevec,m,c0)
    Theta_res = result['Theta'] # vector to add row by row
    ThetaM.append(Theta_res)
    zeta_res = result['zeta']
    zetaM.append(zeta_res)
    #C_res = result['C_index']
    g_test_res = result['g_test']
    G_test_deep.append(result['g_test']) # vector to add row by row
Error_dcp = np.mean(np.array(G_test_deep), axis=0) - g_true
Theta_dcp = np.mean(np.array(ThetaM),axis=0)
zeta_dcp = np.mean(np.array(zeta_res))
print(Error_dcp)
print(Theta_dcp)
print(zeta_dcp)